In [1]:
%cd ..

# imports:
# todo: hide/combine/simplify these imports
from domblar.sc3.client import SC3Client
from domblar.sc3.instruments import setup_instruments, assign_instrument
from domblar.players import play, play_voices
from domblar.transformations import iterate_voice_leadings, shift, transpose, split
from domblar.rhythm import get_grouped_rhythm, rhythm_to_notes
from domblar.edo import build_mos_from_edo, intervals_in_edo

/Users/gexahedron/devel/domblar


In [2]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell

# 2. run this cell
client = SC3Client()
instruments = setup_instruments(client, 4, 'Dexed.vst3', 'dexed_preset')

In [3]:
# play:

def assign_instruments():
    # assign_instrument(0, 'tub_bells', instruments, client)
    assign_instrument(0, 'brass/toto_africa', instruments, client)
assign_instruments()

edo = 37
# fixme: remove this scale, looks redundant
scale = list(range(edo))

scales = {}
scales['orgone7'] = build_mos_from_edo(7, gen=10, edo=edo, down=0, shift=0) + [edo]

cur_scale = scales['orgone7']
print(cur_scale)
print('steps -', intervals_in_edo(cur_scale, edo))
play(cur_scale, scale, edo, client, synth_idx=[0], dur=0.3)


[0, 3, 10, 13, 20, 23, 30, 37]
steps - [3, 7, 3, 7, 3, 7, 7]


In [5]:
chords = iterate_voice_leadings(
    [0, 2, 7, 11],
    [-2, 10, -2, -2],
    [0, 2, 3, 1],
    7)
print(chords)

def assign_instruments():
    for i in range(4):
        # assign_instrument(i, 'tub_bells', instruments, client)
        assign_instrument(i, 'brass/toto_africa', instruments, client)
assign_instruments()

edo = 12
scale = list(range(edo))
play(chords, scale, edo, client, synth_idx=[0, 1, 2, 3], dur=0.75)


[[0, 2, 7, 11], [-2, 5, 9, 12], [-4, 7, 10, 15], [-6, 8, 13, 17], [-8, 11, 15, 18], [-10, 13, 16, 21], [-12, 14, 19, 23]]


In [6]:
# melody example
# Stephane Picq, Dune - Water
# what i like about this is
# - simplicity / repetitiveness
# - nice modulations
# values below are MIDI values

def arpeggio_dune_water(notes, reps=2):
    assert (len(notes) == 4)
    notes.sort()
    v1 = [notes[3], notes[2], notes[1], notes[2]]
    v2 = [notes[3], notes[2], notes[0], notes[2]]
    return (v1 + v2) * reps

# c3 = 48; melody starts on d#3, 51, it's d# lydian actually
# vibraphone
part1 = arpeggio_dune_water([43, 45, 46, 51])
part2 = arpeggio_dune_water([43, 45, 46, 50])
part3 = arpeggio_dune_water([43, 45, 47, 52])
part4 = arpeggio_dune_water([47, 49, 50, 54])
# part1, part2, part3, part4 are repeated again

# 1: 51 d# lydian
# 2: 50 d aeolian
# 3: 52 e aeolian
# 4: 47 b aeolian = 54 phrygian (i think i perceive this as aeolian though)

part5 = arpeggio_dune_water([50, 52, 53, 57])
part6 = arpeggio_dune_water([53, 55, 56, 60])
part7 = arpeggio_dune_water([45, 47, 48, 52])
# part8 is a final part and is more tricky:
part8 = arpeggio_dune_water([44, 47, 48, 51], reps=1) + arpeggio_dune_water([44, 46, 48, 51], reps=1)

# 5: 50 d aeolian (part4 shifted by 3 steps)
# 6: 53 f aeolian (part5 shifted by 3 steps)
# 7: 45 a aeolian (part6 shifted down by 8 steps)
# 8: no single (Pressing scale) collection, because 2 consecutive semitones
# 8.1: e harm maj, c harm min, oct(0,2), hex(0,3)
# 8.2: 44 g# ionian

def assign_instruments():
    for i in range(2):
        assign_instrument(i, 'vibraphone/dune_water', instruments, client)
assign_instruments()

edo = 12
scale = list(range(edo))

melody = (part1 + part2 + part3 + part4) * 2 + part5 + part6 + part7 + part8
melody = transpose(melody, -60)  # because i used MIDI values above
melody = split(melody)
melody = melody[:-1] + [['.', '.']]
play(melody, scale, edo, client, synth_idx=[0, 1], dur=0.25)

# TODO: notes are not released completely - we need to cut the release values

KeyboardInterrupt: 

In [38]:
# four on the floor
edo = 2
scale = list(range(edo))

# TODO: replace with 'x...' * 4
kick_voice = ['x', '.', '.', '.'] * 4
kick_voice = rhythm_to_notes(kick_voice, 0)
print(kick_voice)
snare_voice = (['.'] * 4 + ['x', '.', '.', '.']) * 2
snare_voice = rhythm_to_notes(snare_voice, -4*edo)
print(snare_voice)
closed_hihat_voice = ['.'] * 8 + ['.', 'x', '.', '.'] + ['.'] * 4
closed_hihat_voice = rhythm_to_notes(closed_hihat_voice, -edo)
print(closed_hihat_voice)
open_hihat_voice = ['.', '.', 'x', '.'] * 4
open_hihat_voice = rhythm_to_notes(open_hihat_voice, -8*edo)
print(open_hihat_voice)
voices = list(zip(kick_voice, snare_voice, closed_hihat_voice, open_hihat_voice))

def assign_instruments():
    assign_instrument(0, 'timpani2', instruments, client)
    assign_instrument(1, 'snare1', instruments, client)
    assign_instrument(2, 'closed_hi_hat', instruments, client)
    assign_instrument(3, 'hi_hat', instruments, client)
assign_instruments()

for _ in range(4):
    play(voices, scale, edo, client, synth_idx=[0, 1, 2, 3], dur=0.12)


[0, '.', '.', '.', 0, '.', '.', '.', 0, '.', '.', '.', 0, '.', '.', '.']
['.', '.', '.', '.', -8, '.', '.', '.', '.', '.', '.', '.', -8, '.', '.', '.']
['.', '.', -16, '.', '.', '.', -16, '.', '.', '.', -16, '.', '.', '.', -16, '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', -16, '.', '.', '.', '.', '.', '.']


In [55]:
# the levee break
edo = 2
scale = list(range(edo))

kick_voice = ['x', 'x', '.', '.'] + ['.', '.', '.', 'x'] + ['.', '.', 'x', 'x'] + ['.'] * 4
kick_voice = rhythm_to_notes(kick_voice, 0)
print(kick_voice)
snare_voice = (['.'] * 4 + ['x', '.', '.', '.']) * 2
snare_voice = rhythm_to_notes(snare_voice, edo)
print(snare_voice)
closed_hihat_voice = ['x', '.'] * 8
closed_hihat_voice = rhythm_to_notes(closed_hihat_voice, -8*edo)
print(closed_hihat_voice)
voices = list(zip(kick_voice, snare_voice, closed_hihat_voice))

def assign_instruments():
    assign_instrument(0, 'timpani2', instruments, client)
    assign_instrument(1, 'log_drum', instruments, client)
    assign_instrument(2, 'closed_hi_hat', instruments, client)
assign_instruments()

for _ in range(4):
    play(voices, scale, edo, client, synth_idx=[0, 1, 2], dur=0.15)


[0, 0, '.', '.', '.', '.', '.', 0, '.', '.', 0, 0, '.', '.', '.', '.']
['.', '.', '.', '.', 2, '.', '.', '.', '.', '.', '.', '.', 2, '.', '.', '.']
[16, '.', 16, '.', 16, '.', 16, '.', 16, '.', 16, '.', 16, '.', 16, '.']


KeyboardInterrupt: 

In [ ]:
# 12edo modulation between scales


In [ ]:
# 12edo Neo-Riemannian transformations

In [ ]:
# jazz examples


In [ ]:
# 12edo scales examples
# high register chords, bell-like
# George Crumb: Music for a Summer Evening - V. Music of the Starry Night
# https://www.youtube.com/watch?v=jTLq-9xyPz4&ab_channel=ZhengchaoDing
# Romitelli - Amok Koma
# https://www.youtube.com/watch?v=8tAWPIFD1_w&ab_channel=SchallfeldEnsemble
# Desyatnikov - Kolkhoz Song About Moscow
# https://www.youtube.com/watch?v=bBaIEapbI9o&ab_channel=VariousArtists-Topic


In [ ]:
# pentatonic planing in Debussy - La Cathédrale Engloutie
# https://www.youtube.com/watch?v=yTwvtwlSh5U

In [ ]:
# varying dynamics example/study
# canto ostinato, section 31
# https://www.youtube.com/watch?v=1Zk1xyrWfX0


In [ ]:
# augmented sixth examples


In [ ]:
# maqamat


In [ ]:
# dynamic tonality

In [13]:
# cleanup
# TODO
# do stop_server()
